In [1]:
import robotic as ry
import time
import numpy as np
from thrower_generic import find_velocity, pick_last_object_if_valid, grasp_object, throw_object, check_in_the_bin

In [2]:
def point_cloud(C: ry.Config, obj: ry.Frame):
    obj_xsize, obj_ysize, obj_zsize, obj_dim = obj.getSize()
    obj_xcenter, obj_ycenter, obj_zcenter = obj.getPosition()

    obj_xmin, obj_xmax = obj_xcenter - obj_xsize / 2, obj_xcenter + obj_xsize / 2
    obj_ymin, obj_ymax = obj_ycenter - obj_ysize / 2, obj_ycenter + obj_ysize / 2
    obj_zmin, obj_zmax = obj_zcenter - obj_zsize / 2, obj_zcenter + obj_zsize / 2

    # Camera list
    camera_names = ['camera1', 'camera2', 'camera3', 'camera4']
    all_pcl_world = []  # To store point clouds from all cameras

    for cam_name in camera_names:
        # Set up the camera
        cam = ry.CameraView(C)
        cam.setCamera(cam_name)
        rgb, depth = cam.computeImageAndDepth(C)
        pcl_camera = ry.depthImage2PointCloud(depth, cam.getFxycxy())  # (height, width, 3)

        # Flatten the point cloud
        pcl_flat = pcl_camera.reshape(-1, 3)

        # Get camera transformation (position and rotation)
        camera_frame = C.getFrame(cam_name)
        cam_pos = camera_frame.getPosition()  # Translation (3,)
        cam_rot = camera_frame.getRotationMatrix()  # Rotation (3x3)

        # Construct the transformation matrix (4x4)
        camera_transform = np.eye(4)
        camera_transform[:3, :3] = cam_rot  # Rotation part
        camera_transform[:3, 3] = cam_pos  # Translation part

        # Transform the point cloud to the world frame
        ones = np.ones((pcl_flat.shape[0], 1))  # Add a fourth homogeneous coordinate
        pcl_homogeneous = np.hstack([pcl_flat, ones])  # (N, 4)
        pcl_world_homogeneous = pcl_homogeneous @ camera_transform.T  # Transform (N, 4)
        pcl_world = pcl_world_homogeneous[:, :3]  # Drop the homogeneous coordinate

        # Append transformed point cloud to the global list
        all_pcl_world.append(pcl_world)

    # Merge all point clouds into one
    merged_pcl_world = np.vstack(all_pcl_world)

    # Mask points to include only those inside the obj
    obj_mask = (merged_pcl_world[:, 0] >= obj_xmin ) & (merged_pcl_world[:, 0] <= obj_xmax ) & \
               (merged_pcl_world[:, 1] >= obj_ymin ) & (merged_pcl_world[:, 1] <= obj_ymax ) & \
               (merged_pcl_world[:, 2] >= obj_zmin ) & (merged_pcl_world[:, 2] <= obj_zmax )

    # Apply the mask to get object points
    obj_pcl = merged_pcl_world[obj_mask]

    # Create a frame for the object point cloud and display it
    f = C.addFrame('obj_pcl', 'world')  # Can use any camera here for visualization
    f.setPointCloud(obj_pcl, [0, 255, 0])  # Display the point cloud in red

    C.view()  # View the scene with the object point cloud


In [3]:
def throw_catch(isRender:bool,sleep_time:float = 20):
    C = ry.Config()
    C.addFile("catching.g")
    print(f"Initial bin pos:{C.getFrame('bin').getPosition()}")
    bin_new_position = C.getFrame('bin').getPosition()

    qHome = C.getJointState()
    C.getFrame('cargo').unLink() # to remove a bug
    C.addFrame("throw").setPosition([0,1.8,1.2]).setShape(ry.ST.marker,[.3])
    C.addFrame("release_frame").setPosition([0,0,0]).setShape(ry.ST.marker,[.4]).setColor([1,0,0]).setContact(0)
    C.addFrame("gripper").setPosition(C.getFrame("l2_gripper").getPosition()).setShape(ry.ST.marker,[.3])
    C.addFrame("base").setPosition(C.getFrame("l2_panda_base").getPosition()).setShape(ry.ST.marker,[.3])
    C.view()
    point_cloud(C, C.getFrame('cargo'))

    bot = ry.BotOp(C, useRealRobot=False)
    release_velocity,isInverted = find_velocity(C)
    initial_position = pick_last_object_if_valid(C,C.getFrame("release_frame").getPosition(),release_velocity)
    #invertion deviation calculation
    if isInverted:
        deviation_dist = 0.2
        a = -release_velocity[1]/release_velocity[0]
        dev_x = deviation_dist/np.sqrt(1+np.square(a))
        dev_y = deviation_dist/np.sqrt(1+np.square(1/a))
        deviation_arr = np.array([dev_x,dev_y,0])
        time_deviation = grasp_object(C,bot,isInverted,deviation_arr)
    else:
        time_deviation = grasp_object(C,bot,isInverted)
    print(f"Final bin pos:{C.getFrame('bin').getPosition()}")
    wanted_sleep = 0.5
    time_sleep = time_deviation * wanted_sleep
    throw_object(C,bot,time_sleep,release_velocity)
    cargo_height = C.getFrame("cargo").getSize()[2]
    result, deviation = check_in_the_bin(C,bot,bin_new_position,C.getFrame("side2").getSize()[0]/2,C.getFrame("side2").getSize()[2],cargo_height)

    print(f"Result:{result} for bin pos:{bin_new_position}")

    if isRender:
        C.view()
        time.sleep(sleep_time)
    del C
    del bot
    return result, deviation

In [4]:
throw_catch(True,sleep_time=10)

Initial bin pos:[1.71746677 2.0591233  0.52644299]
-- WARNING:kin.cpp:addFrame:193(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:getFrame:412(-1) cannot find frame named 'world'
Initial Position: [0.  1.8 1.2]
Bin Position: [1.71746677 2.0591233  0.52644299]
constraint_x: x_land=0.553061164752, x_bin=1.7174667716448975
constraint_y: y_land=1.8190389721382276, y_bin=2.0591233015399677
constraint_z: z_land=1.6332205369570056, z_bin=0.5264429903525961
constraint_x: x_land=0.553061164752, x_bin=1.7174667716448975
constraint_y: y_land=1.8190389721382276, y_bin=2.0591233015399677
constraint_z: z_land=1.6332205369570056, z_bin=0.5264429903525961
constraint_x: x_land=0.5530611652510832, x_bin=1.7174667716448975
constraint_y: y_land=1.8190389721554083, y_bin=2.0591233015399677
constraint_z: z_land=1.633220537955763, z_bin=0.5264429903525961
constraint_x: x_land=0.5530611507649078, x_bin=1.7174667716448975
constraint_y: y_land=1.819038971656726, y_bin=2.0

(False, np.float64(1.9397238252989863))